## House Price Prediction with MLFLOW ##

In this project we will:
+ Run a hyperparameter tuning while training a model
+ Log every hyperparamater and metrics in the MLFLOW UI
+ Compare the results of the various runs in the MLFLOW UI
+ Choose the best run and register it as a model

In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [2]:
## Prepering Dataset

data = pd.DataFrame(housing.data,columns=housing.feature_names)
data['Price'] = housing.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


# Train test split, Model Hyperparameter Tuning, MLFlOW Experiments #

In [3]:
from urllib.parse import urlparse
## Divide data into Independent and dependent features
X = data.drop(columns=['Price'])
y = data['Price']

In [4]:
## Hyperparamter tuning using GridSearchCV

def hyperparamter_tuning(X_train,y_train,param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf,param_grid=param_grid,cv=3,n_jobs=-1,verbose=2,scoring="neg_mean_squared_error")
    grid_search.fit(X_train,y_train)
    return grid_search

In [5]:
## Split the data into training and testing sets

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20)

from mlflow.models import infer_signature
signature = infer_signature(X_train,y_train)

##Define the hyperparametrs grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5,10, None],
    'min_samples_split': [2,5],
    'min_samples_leaf': [1,2]
}

## Start the mlflow experiment
with mlflow.start_run():
    ## Performing hyperparameter tuning
    grid_search = hyperparamter_tuning(X_train,y_train,param_grid)

    ## get the best model with best params
    best_model = grid_search.best_estimator_

    #Evaluate the best model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)

    ## Log the best parameter and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse", mse)

    ## Tracking URI

    mlflow.set_tracking_uri(uri = "https://127.0.0.1:5000")  ## This is done when we are tracking using remote server tracking UI
    tracking_url_type_store = urlparse( mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model,"model",registered_model_name="BEst random forest model")
    else:
        mlflow.sklearn.log_model(best_model,"model",signature=signature)
        ## we have set the signature if the tracking is done using files
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/04/19 11:17:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MlflowException: API request to https://127.0.0.1:5000/api/2.0/mlflow/runs/get failed with exception HTTPSConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/get?run_uuid=b86d1c10b282492a8ec148e932d2311f&run_id=b86d1c10b282492a8ec148e932d2311f (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000014DD54D86B0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))